In [52]:
# importing standard packages
import pandas as pd
import numpy as np 
from scipy import stats 

# importing the plot funnctions
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline 

# preprocessing/ model selection 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# importing the classifiers 
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

# importing the metrics 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, classification_report,f1_score
from sklearn.metrics import roc_curve

# oversampling techniques 
from imblearn.over_sampling import SMOTE

# importing model saving package 
from joblib import dump, load


## Functions for Notebooks

In [53]:
def draw_conf_mat(mat):
    
    """
    Draw confusion matrix
    
    Parameters:
    ------------------
    mat:                ndarray of shape (n_classes, n_classes) 
                        confusion matrix

    """
    
    import matplotlib.pyplot as plt 
    import seaborn as sns
    
    fig, ax = plt.subplots(figsize=(10, 10))
    sns.heatmap(mat, annot=True, fmt="d",
            xticklabels=['0', '1'],
            yticklabels=['0', '1'])
    plt.ylabel("Actual")
    plt.xlabel("Predicted")
    
    return ax

In [54]:
def draw_roc_curve(y_true, y_proba):
    
    """
    Draw baseline and model roc curve
    
    Parameters:
    ------------------
    y_true:             array-like of shape (n_samples,) 
                        True label of target (y)
                        
    y_proba             array-like of shape (n_samples,)
                        The predicted probability of target (y)
    """
    
    
    base_fpr, base_tpr, _ = roc_curve(y_true, [1 for _ in range(len(y_true))])
    model_fpr, model_tpr, _ = roc_curve(y_true, y_proba)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    plt.plot(base_fpr, base_tpr, 'b', label = 'baseline')
    plt.plot(model_fpr, model_tpr, 'r', label = 'model')
    plt.legend();
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate'); plt.title('ROC Curves');
    
    return ax

In [55]:
def metrics_imbalanced(your_confusion_matrix, y_true, y_proba):
    
    """
    Print precision, recall, fallout and auroc score based on the confusion matrix. 
    
    Parameters:
    ------------------
    your_confusion_matrix:                ndarray of shape (n_classes, n_classes) 
                                          confusion matrix
                                          
    y_true:                               array-like of shape (n_samples,) 
                                          True label of target (y)
                                          
    y_proba:                              y_proba             array-like of shape (n_samples,)
                                          The predicted probability of target (y)
    """
    
    # Model evaluation metrics. 
    tp = your_confusion_matrix[1,1]
    fn = your_confusion_matrix[1,0]
    fp = your_confusion_matrix[0,1]
    tn = your_confusion_matrix[0,0]
    auroc = roc_auc_score(y_true, y_proba)
    
    print('Precision = %0.3f'%(tp/(tp+fp)))
    print('Recall (TPR) = %0.3f'%(tp/(tp+fn)))
    print('Fallout (FPR) = %0.3f'%(fp/(fp+tn)))
    print('Roc_auc_score = %0.3f'%auroc)

In [ ]:
def resampling_unbalanced(train_x, train_y, sample_method):
    """ 

    Parameters
    ----------
    train_x : test_x 
            pd.DataFrame
    train_y : train_y 
            pd.DataFrame
    sample_method: 'over' or 'under'
            string

    Returns
    -------
    train_x : resampled train_x
                pd.DataFrame
    train_y : resampled train_y
                pd.DataFrame
        

    """
    
    
    if sample_method == "over":
        oversample = SMOTE()
        train_x_ros, train_y_ros = oversample.fit_resample(X_train, y_train)
        
        return train_x_ros, train_y_ros
        
    if sample_method == "under":
        # Concate X and Y train
        trainData = pd.concat([train_x, train_y],axis=1)
        # Class count
        count_class_0, count_class_1 = trainData["hospital_death"].value_counts()

        # Divide by class
        df_class_0 = trainData[trainData['hospital_death'] == 0]
        df_class_1 = trainData[trainData['hospital_death'] == 1]
        
        # Sample the majority class
        df_class_0_under = df_class_0.sample(count_class_1)
        
        # Put the train dataset together
        train_rus = pd.concat([df_class_0_under, df_class_1], axis=0)
        train_x_rus = train_rus.drop("hospital_death", axis = 1)
        train_y_rus = pd.DataFrame(train_rus['hospital_death'])
        
        return train_x_rus, train_y_rus

## Bring in the X_wids and y_wids from the WIDS_Feature_Engineer.ipynb
Run the WIDS Feature Engineer Notebook first in order to grab the dataframes

In [ ]:
# grab the stored data frames
%store -r X_wids
%store -r y_wids

In [ ]:
X_wids.shape, y_wids.shape

((91713, 598), (91713, 1))

## Train Test Split without Correction of Imbalanced Data

In [ ]:
# Split train-test dataset 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_wids, y_wids, test_size = 0.2, random_state = 31, stratify = y_wids)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(73370, 598) (18343, 598) (73370, 1) (18343, 1)


## Models with No Correction Of Imbalanced Data

In [ ]:
# create a dictionary of classifers with the hyperparameters you would like to try 
classifiers = {'XG': XGBClassifier(n_jobs = 1, learning_rate = .1, n_estimators = 100, objective = 'binary:hinge', booster='gbtree', max_depth = 7 ),
              'RF': RandomForestClassifier(n_estimators= 100, max_depth = 7),
              'LGB': LGBMClassifier(booster ='gbtree', n_estimators = 100, learning_rate = .1, max_depth = 7, objective ='binary')}

In [ ]:
# keys of the classifiers 
selected_clfs = ['XG','RF','LGB']

In [ ]:
# create the clf_matrix 
clf_matrix = pd.DataFrame(columns = ['model', 'precision_test', 'recall_test', 'FPR_test', 'AUROC_test',
                                     'f1_test', 'f1_train', 'f1_CV'])

# for every classifer in the selected classifers list
for idx, classifier in enumerate(selected_clfs):

    # get the classifer and hyperparameters from the model
    clf = classifiers[classifier]

    # fit the model 
    clf.fit(X_train,y_train.values.ravel())
    print(f"-------- classifier being run is {classifier} ---------")

    print(f"-------- saving the the model ---------")
    # Write the model to file
    dump(clf, '{}.joblib'.format(classifier)) 

    print(f"-------- computing y_predict, y_prob, y_train_predict ---------")
    # prediction of y based on X_test
    y_predict = clf.predict(X_test)
    # prediction of y probability based on X_test
    y_proba = clf.predict_proba(X_test)[:,1]
    # prediction of y based on X_train
    y_train_predict = clf.predict(X_train)

    print(f"-------- creating the confusion matrix ---------") 
    # confusion matrix 
    cmat = confusion_matrix(y_test.values.ravel(), y_predict)
    # tp, fn, fp, tn
    tp = cmat[1,1]
    fn = cmat[1,0]
    fp = cmat[0,1]
    tn = cmat[0,0]

    print(f"-------- calculating the metrics ---------") 
    #precision score on test
    p_score_test = precision_score(y_test.values.ravel(), y_predict)

    #recall score on test
    r_score_test = tp/(tp+fn) 

    #FPR score on test
    fpr_test = fp/(fp+tn)

    #AUROC score on test
    auroc_test = roc_auc_score(y_test.values.ravel(), y_proba)

    #f1 score on test
    f1_test = f1_score(y_test.values.ravel(), y_predict)

    #f1 score on train
    f1_train = f1_score(y_train.values.ravel(), y_train_predict)

    #f1 score on cross validation 
    k_fold = 4
    f1_cv = cross_val_score(clf, X_train, y_train.values.ravel(), cv=k_fold, scoring = 'f1')
    print(f"Cross Validation is on {k_fold} folds.")

    print(f"-------- append values to the model matrix  ---------") 

    # append to matrix
    df2 = pd.DataFrame([[classifier,p_score_test,r_score_test, fpr_test, auroc_test, f1_test, f1_train, f1_cv]], 
                       columns=['model', 'precision_test', 'recall_test', 'FPR_test', 'AUROC_test',
                                     'f1_test', 'f1_train', 'f1_CV'])
    
    clf_matrix = pd.concat([df2, clf_matrix])


-------- classifier being run is XG ---------
-------- saving the the model ---------
-------- computing y_predict, y_prob, y_train_predict ---------
-------- creating the confusion matrix ---------
-------- calculating the metrics ---------


In [ ]:
# Draw confusion matrix for each model 
for classifier in selected_clfs:

  # Load model saved
  clf = load('{}.joblib'.format(classifier)) 

  # Predict y based on X_test
  y_predict = clf.predict(X_test)

  # confusion matrix
  cm = confusion_matrix(y_test, y_predict)

  # Draw
  print(classifier)
  draw_conf_mat(cm)


## Models with Correction Of Imbalanced Data Using Undersampling
Note: Undersampling should not be performed on the validation set. Therefore we will build a pipeline of undersampling and modeling. 


In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline, make_pipeline

In [ ]:
# create a dictionary of classifers with the hyperparameters you would like to try 
classifiers = {'XG_U': XGBClassifier(n_jobs = 1, learning_rate = .1, n_estimators = 100, objective = 'binary:hinge', booster='gbtree', max_depth = 7 ),
              'RF_U': RandomForestClassifier(n_estimators= 100, max_depth = 7),
              'LGB_U': LGBMClassifier(booster ='gbtree', n_estimators = 100, learning_rate = .1, max_depth = 7, objective ='binary')}

In [ ]:
# keys for the classifiers 
selected_clfs = [ 'XG_U', 'RF_U', 'LGB_U']

Undersampling data using 0.8 ratio (minority:majority)

In [ ]:
clf_under_matrix = pd.DataFrame(columns = ['model', 'precision_test', 'recall_test', 'FPR_test', 'AUROC_test',
                                     'f1_test', 'f1_train', 'f1_CV', 'recall_CV'])

# for every classifer in the selected classifers list
for idx, classifier in enumerate(selected_clfs):

    # get the classifer and hyperparameters from the model
    clf = classifiers[classifier]
    
    # Make pipeline
    rus = RandomUnderSampler(sampling_strategy = 0.8, random_state=42)
    clf_rus_pipeline = make_pipeline(rus,clf)
    
    # fit the model 
    clf_rus_pipeline.fit(X_train, y_train.values.ravel())
    print(f"-------- classifier being run is {classifier} ---------")

    print(f"-------- saving the the model ---------")
    # Write the model/pipeline to file
    dump(clf_rus_pipeline, '{}.joblib'.format(classifier)) 

    print(f"-------- computing y_predict, y_prob, y_train_predict ---------")
    # prediction of y based on X_test. RUS does not do anything to X_test. 
    y_predict = clf_rus_pipeline.predict(X_test)
    # prediction of y probability based on X_test
    y_proba = clf_rus_pipeline.predict_proba(X_test)[:,1]
    # prediction of y based on X_train
    y_train_predict = clf_rus_pipeline.predict(X_train)

    print(f"-------- creating the confusion matrix ---------") 
    # confusion matrix 
    cmat = confusion_matrix(y_test.values.ravel(), y_predict)
    # tp, fn, fp, tn
    tp = cmat[1,1]
    fn = cmat[1,0]
    fp = cmat[0,1]
    tn = cmat[0,0]

    print(f"-------- calculating the metrics ---------") 
    #precision score on test
    p_score_test = precision_score(y_test.values.ravel(), y_predict)

    #recall score on test
    r_score_test = tp/(tp+fn) 

    #FPR score on test
    fpr_test = fp/(fp+tn)

    #AUROC score on test
    auroc_test = roc_auc_score(y_test.values.ravel(), y_proba)

    #f1 score on test
    f1_test = f1_score(y_test.values.ravel(), y_predict)

    #f1 score on train
    f1_train = f1_score(y_train.values.ravel(), y_train_predict)

    #f1 score on cross validation 
    k_fold = 4
    f1_cv = cross_val_score(clf_rus_pipeline, X_train, y_train.values.ravel(), 
                            cv=k_fold, scoring = 'f1')
    recall_cv = cross_val_score(clf_rus_pipeline, X_train, y_train.values.ravel(), 
                            cv=k_fold, scoring = 'recall')
    print(f"Cross Validation is on {k_fold} folds.")

    print(f"-------- append values to the model matrix  ---------") 

    # append to matrix
    df2 = pd.DataFrame([[classifier,p_score_test,r_score_test, fpr_test, auroc_test, f1_test, f1_train, f1_cv, recall_cv]], 
                       columns=['model', 'precision_test', 'recall_test', 'FPR_test', 'AUROC_test',
                                     'f1_test', 'f1_train', 'f1_CV', 'recall_CV'])
    
    clf_under_matrix = pd.concat([df2, clf_under_matrix])


In [ ]:
clf_under_matrix

In [ ]:
# Draw confusion matrix for each model 
for classifier in selected_clfs:

  # Load model saved
  clf = load('{}.joblib'.format(classifier)) 

  # Predict y based on X_test
  y_predict = clf.predict(X_test)

  # confusion matrix
  cm = confusion_matrix(y_test, y_predict)

  # Draw
  print(classifier)
  draw_conf_mat(cm)

In [ ]:
clf_under_matrix

## Models with Correction Of Imbalanced Data Using Oversampling SMOTE
Again, smote needs to be performed outside of validation set. Make pipeline first.

In [ ]:
# create a dictionary of classifers with the hyperparameters you would like to try 
classifiers = {'XG_O': XGBClassifier(n_jobs = 1, learning_rate = .1, n_estimators = 100, objective = 'binary:hinge', booster='gbtree', max_depth = 7 ),
              'RF_O': RandomForestClassifier(n_estimators= 100, max_depth = 7),
              'LGB_O': LGBMClassifier(booster ='gbtree', n_estimators = 100, learning_rate = .1, max_depth = 7, objective ='binary')}

In [ ]:
selected_clfs = ['XG_O', 'RF_O', 'LGB_O' ]

Oversampling using 0.8 ratio as well

In [ ]:
clf_over_matrix = pd.DataFrame(columns = ['model', 'precision_test', 'recall_test', 'FPR_test', 'AUROC_test',
                                     'f1_test', 'f1_train', 'f1_CV', 'recall_cv'])

# for every classifer in the selected classifers list
for idx, classifier in enumerate(selected_clfs):
    
    # get the classifer and hyperparameters from the model
    clf = classifiers[classifier]
    
    # Make pipeline
    ros = SMOTE(sampling_strategy = 0.8, random_state=42)
    clf_ros_pipeline = make_pipeline(ros,clf)

    # fit the model 
    clf_ros_pipeline.fit(X_train,y_train.values.ravel())
    print(f"-------- classifier being run is {classifier} ---------")

    print(f"-------- saving the the model ---------")
    # Write the model to file
    dump(clf_ros_pipeline, '{}.joblib'.format(classifier)) 

    print(f"-------- computing y_predict, y_prob, y_train_predict ---------")
    # prediction of y based on X_test
    y_predict = clf_ros_pipeline.predict(X_test)
    # prediction of y probability based on X_test
    y_proba = clf_ros_pipeline.predict_proba(X_test)[:,1]
    # prediction of y based on X_train
    y_train_predict = clf_ros_pipeline.predict(X_train)

    print(f"-------- creating the confusion matrix ---------") 
    # confusion matrix 
    cmat = confusion_matrix(y_test.values.ravel(), y_predict)
    # tp, fn, fp, tn
    tp = cmat[1,1]
    fn = cmat[1,0]
    fp = cmat[0,1]
    tn = cmat[0,0]

    print(f"-------- calculating the metrics ---------") 
    #precision score on test
    p_score_test = precision_score(y_test.values.ravel(), y_predict)

    #recall score on test
    r_score_test = tp/(tp+fn) 

    #FPR score on test
    fpr_test = fp/(fp+tn)

    #AUROC score on test
    auroc_test = roc_auc_score(y_test.values.ravel(), y_proba)

    #f1 score on test
    f1_test = f1_score(y_test.values.ravel(), y_predict)

    #f1 score on train
    f1_train = f1_score(y_train.values.ravel(), y_train_predict)

    #f1 score on cross validation 
    k_fold = 4
    f1_cv = cross_val_score(clf_ros_pipeline, X_train, y_train.values.ravel(), 
                            cv=k_fold, scoring = 'f1')
    recall_cv = cross_val_score(clf_ros_pipeline, X_train, y_train.values.ravel(), 
                            cv=k_fold, scoring = 'recall')
    print(f"Cross Validation is on {k_fold} folds.")

    print(f"-------- append values to the model matrix  ---------") 

    # append to matrix
    df2 = pd.DataFrame([[classifier,p_score_test,r_score_test, fpr_test, auroc_test, f1_test, f1_train, f1_cv, recall_cv]], 
                       columns=['model', 'precision_test', 'recall_test', 'FPR_test', 'AUROC_test',
                                     'f1_test', 'f1_train', 'f1_CV', 'recall_cv'])
    
    clf_over_matrix = pd.concat([df2, clf_over_matrix])


In [ ]:
# Draw confusion matrix for each model 
for classifier in selected_clfs:

  # Load model saved
  clf = load('{}.joblib'.format(classifier)) 

  # Predict y based on X_test
  y_predict = clf.predict(X_test)

  # confusion matrix
  cm = confusion_matrix(y_test, y_predict)

  # Draw
  print(classifier)
  draw_conf_mat(cm)

In [33]:
clf_over_matrix.explode('f1_CV')

,model,precision_test,recall_test,FPR_test,AUROC_test,f1_test,f1_train,f1_CV,recall_cv
0,LGB_O,0.681537,0.347442,0.015334,0.896934,0.460251,0.538709,0.449364,"[0.3461781427668983, 0.3581806696146557, 0.344..."
0,LGB_O,0.681537,0.347442,0.015334,0.896934,0.460251,0.538709,0.474477,"[0.3461781427668983, 0.3581806696146557, 0.344..."
0,LGB_O,0.681537,0.347442,0.015334,0.896934,0.460251,0.538709,0.457286,"[0.3461781427668983, 0.3581806696146557, 0.344..."
0,LGB_O,0.681537,0.347442,0.015334,0.896934,0.460251,0.538709,0.46606,"[0.3461781427668983, 0.3581806696146557, 0.344..."
0,RF_O,0.365369,0.581175,0.095346,0.857705,0.448671,0.461295,0.431449,"[0.5672773215413771, 0.5874921036007581, 0.579..."
0,RF_O,0.365369,0.581175,0.095346,0.857705,0.448671,0.461295,0.451161,"[0.5672773215413771, 0.5874921036007581, 0.579..."
0,RF_O,0.365369,0.581175,0.095346,0.857705,0.448671,0.461295,0.458868,"[0.5672773215413771, 0.5874921036007581, 0.579..."
0,RF_O,0.365369,0.581175,0.095346,0.857705,0.448671,0.461295,0.442221,"[0.5672773215413771, 0.5874921036007581, 0.579..."
0,LGB_O,0.681537,0.347442,0.015334,0.896934,0.460251,0.538709,0.449364,"[0.3461781427668983, 0.3581806696146557, 0.344..."
0,LGB_O,0.681537,0.347442,0.015334,0.896934,0.460251,0.538709,0.474477,"[0.3461781427668983, 0.3581806696146557, 0.344..."


In [70]:
clf_under_matrix.explode('f1_CV')

,model,precision_test,recall_test,FPR_test,AUROC_test,f1_test,f1_train,f1_CV
0,LGB_U,0.289626,0.821857,0.190394,0.899827,0.428313,0.913517,0.799745
0,LGB_U,0.289626,0.821857,0.190394,0.899827,0.428313,0.913517,0.819072
0,LGB_U,0.289626,0.821857,0.190394,0.899827,0.428313,0.913517,0.818754
0,LGB_U,0.289626,0.821857,0.190394,0.899827,0.428313,0.913517,0.805276
0,RF_U,0.294895,0.733418,0.165632,0.868802,0.420652,0.801756,0.75663
0,RF_U,0.294895,0.733418,0.165632,0.868802,0.420652,0.801756,0.781022
0,RF_U,0.294895,0.733418,0.165632,0.868802,0.420652,0.801756,0.773873
0,RF_U,0.294895,0.733418,0.165632,0.868802,0.420652,0.801756,0.771259
0,XG_U,0.273634,0.797220,0.199881,0.798670,0.407425,0.957412,0.791481
0,XG_U,0.273634,0.797220,0.199881,0.798670,0.407425,0.957412,0.798984


In [32]:
clf_matrix.explode('f1_CV')

NameError: name 'clf_matrix' is not defined

In [29]:
## Warnings seen on terminal

### [LightGBM] [Warning] Unknown parameter: booster
### [LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves

### Feature Importance

In [72]:
lgb = load("LGB.joblib")
baseline_importances = pd.DataFrame({'feature':X_wids.columns,'importance':np.round(lgb.feature_importances_,3)})
baseline_importances = baseline_importances.sort_values('importance',ascending=False).set_index('feature')
baseline_importances.head(600)

,importance
feature,
age,82
d1_heartrate_min,71
pre_icu_los_days,57
d1_resprate_min,51
bmi,49
d1_temp_max,47
d1_glucose_min,42
d1_platelets_min,39
d1_sodium_max,38


In [73]:
lgb_u = load("LGB_U.joblib")
lgb_u_importances = pd.DataFrame({'feature':X_wids.columns,'importance':np.round(lgb_u.feature_importances_,3)})
lgb_u_importances = lgb_u_importances.sort_values('importance',ascending=False).set_index('feature')
lgb_u_importances.head(600)

,importance
feature,
age,85
d1_heartrate_min,51
pre_icu_los_days,45
bmi,45
d1_bun_min,44
d1_resprate_min,43
d1_platelets_min,40
d1_resprate_max,39
d1_spo2_min,39


In [42]:
pd.set_option("display.max_rows", None)

In [75]:
%store X_test
%store y_test

Stored 'X_test' (DataFrame)
Stored 'y_test' (DataFrame)
